In [1]:
#Importing the Data
import pandas as pd
df = pd.read_csv("C:\\Users\\moham\\Desktop\\blogs.csv") 
df


,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [5]:
#  perform an exploratory data analysis to and perform an exploratory data analysis to 

# Check the first few rows of the dataframe
print(df.head())

# Get info about the dataframe, including data types and missing values
print(df.info())

# Describe the numerical features 
print(df.describe())

# Check for missing values
print(df.isnull().sum())

                                                Data       Labels
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....  alt.atheism
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...  alt.atheism
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...  alt.atheism
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
None
                                                     Data       Labels
count                                                2000         2000
unique                                               2000           20
top     Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
freq                    

In [7]:
#Preprocess the data by cleaning the text (removing punctuation, converting to lowercase, etc.), tokenizing, and removing stopwords on both the colums Data and labels

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def preprocess_text(text):
  """
  Preprocesses text by removing punctuation, converting to lowercase, 
  tokenizing, and removing stopwords.
  """
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [w for w in tokens if not w in stop_words]
  return " ".join(filtered_tokens)


# Assuming 'Data' and 'Labels' are the column names in your dataframe
df['Data'] = df['Data'].astype(str).apply(preprocess_text)
df['Labels'] = df['Labels'].astype(str).apply(preprocess_text)


print(df.head())



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                Data      Labels
0  path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism
1  newsgroups altatheism path cantaloupesrvcscmue...  altatheism
2  path cantaloupesrvcscmuedudasnewsharvardedunoc...  altatheism
3  path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism
4  xref cantaloupesrvcscmuedu altatheism53485 tal...  altatheism


In [9]:
# Perform feature extraction to convert text data into a format that can be used by the Naive Bayes model, using techniques such as TF-IDF.

from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the 'Data' column to create a TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(df['Data'])

# Print the shape of the TF-IDF matrix (number of documents x number of features)
print(tfidf_matrix.shape)

# You can access the feature names (words) using vectorizer.get_feature_names_out()
# feature_names = vectorizer.get_feature_names_out() 
# print(feature_names) 

# Now, tfidf_matrix can be used as input for your Naive Bayes model. 


(2000, 56432)


In [11]:
# 	Split the data into training and test sets.

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_matrix, df['Labels'], test_size=0.2, random_state=42
)

# Print the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1600, 56432)
X_test shape: (400, 56432)
y_train shape: (1600,)
y_test shape: (400,)


In [13]:
# Implement a Naive Bayes classifier to categorize the blog posts into their respective categories. You can use libraries like scikit-learn for this purpose
# Train the model on the training set and make predictions on the test set.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Create a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier on the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print a classification report for more detailed evaluation
print(classification_report(y_test, y_pred))


Accuracy: 0.82
                      precision    recall  f1-score   support

          altatheism       0.52      0.89      0.65        18
        compgraphics       0.62      0.83      0.71        18
 composmswindowsmisc       0.95      0.86      0.90        22
compsysibmpchardware       0.95      0.76      0.84        25
  compsysmachardware       0.87      0.95      0.91        21
        compwindowsx       1.00      0.80      0.89        25
         miscforsale       0.92      0.61      0.73        18
            recautos       0.89      0.89      0.89        18
      recmotorcycles       0.88      0.88      0.88        16
    recsportbaseball       0.80      0.89      0.84        18
      recsporthockey       0.83      1.00      0.91        15
            scicrypt       0.82      0.95      0.88        19
      scielectronics       0.68      0.81      0.74        16
              scimed       0.94      0.88      0.91        17
            scispace       1.00      0.86      0.92   

In [15]:
# Choose a suitable library or method for performing sentiment analysis on the blog post texts.

from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
  """
  Analyzes the sentiment of a given text using VADER.
  Returns a sentiment label ('positive', 'negative', 'neutral').
  """
  scores = analyzer.polarity_scores(text)
  compound_score = scores['compound']

  if compound_score >= 0.05:
    return 'positive'
  elif compound_score <= -0.05:
    return 'negative'
  else:
    return 'neutral'

# Apply the get_sentiment function to the 'Data' column of the dataframe
df['Sentiment'] = df['Data'].apply(get_sentiment)

# Print the updated dataframe with sentiment labels
print(df.head())



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                                Data      Labels Sentiment
0  path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism  negative
1  newsgroups altatheism path cantaloupesrvcscmue...  altatheism  positive
2  path cantaloupesrvcscmuedudasnewsharvardedunoc...  altatheism  negative
3  path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism  negative
4  xref cantaloupesrvcscmuedu altatheism53485 tal...  altatheism  positive


In [17]:
# Analyze the sentiments expressed in the blog posts and categorize them as positive, negative, or neutral. Consider only the Data column and get the sentiment for each blog.
# To analyze sentiments, we can examine the distribution of sentiment labels in the 'Sentiment' column:

sentiment_counts = df['Sentiment'].value_counts()
print(sentiment_counts)

# This will provide a count of positive, negative, and neutral sentiments present in the blog posts.

# You can further analyze the sentiment by examining the individual blog posts associated with each sentiment category.
# For example:

positive_blogs = df[df['Sentiment'] == 'positive']
print(positive_blogs.head())

negative_blogs = df[df['Sentiment'] == 'negative']
print(negative_blogs.head())

neutral_blogs = df[df['Sentiment'] == 'neutral']
print(neutral_blogs.head())


Sentiment
positive    1356
negative     581
neutral       63
Name: count, dtype: int64
                                                Data      Labels Sentiment
1  newsgroups altatheism path cantaloupesrvcscmue...  altatheism  positive
4  xref cantaloupesrvcscmuedu altatheism53485 tal...  altatheism  positive
5  xref cantaloupesrvcscmuedu talkabortion120646 ...  altatheism  positive
7  path cantaloupesrvcscmuedudasnewsharvardedunoc...  altatheism  positive
8  xref cantaloupesrvcscmuedu talkabortion120663 ...  altatheism  positive
                                                 Data      Labels Sentiment
0   path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism  negative
2   path cantaloupesrvcscmuedudasnewsharvardedunoc...  altatheism  negative
3   path cantaloupesrvcscmuedumagnesiumclubcccmued...  altatheism  negative
6   newsgroups altatheism path cantaloupesrvcscmue...  altatheism  negative
10  newsgroups altatheism path cantaloupesrvcscmue...  altatheism  negative
       

In [19]:
# Examine the distribution of sentiments across different categories and summarize your findings.

# Group the DataFrame by 'Labels' (categories) and then count the occurrences of each sentiment within each category.
sentiment_by_category = df.groupby(['Labels', 'Sentiment'])['Sentiment'].count().unstack()

# Print the resulting table showing the distribution of sentiments for each category.
print(sentiment_by_category)

# You can also calculate the percentage of each sentiment within each category for a more detailed analysis.
sentiment_by_category_percentage = sentiment_by_category.div(sentiment_by_category.sum(axis=1), axis=0) * 100
print(sentiment_by_category_percentage)

# Summarize the findings based on the distribution table:
print("\nSummary of Sentiment Distribution across Categories:")
for label in sentiment_by_category.index:
  print(f"\nCategory: {label}")
  category_sentiments = sentiment_by_category.loc[label]
  
  # Find the sentiment with the highest count for the current category.
  dominant_sentiment = category_sentiments.idxmax()
  
  print(f"Dominant Sentiment: {dominant_sentiment}")

  # You can further analyze the distribution by providing more context within each category.
  
  # For Example:
  # if dominant_sentiment == 'positive':
  #   print("This category has the highest number of positive sentiments.")
  # elif dominant_sentiment == 'negative':
  #   print("This category has the highest number of negative sentiments.")
  # else:
  #   print("This category has the highest number of neutral sentiments.")

# This analysis will give you a general overview of which sentiments are predominantly expressed within different blog categories, 
# providing insights into the overall tone and feelings associated with each category.


Sentiment             negative  neutral  positive
Labels                                           
altatheism                40.0      1.0      59.0
compgraphics              10.0      3.0      87.0
composmswindowsmisc       22.0      2.0      76.0
compsysibmpchardware      18.0      3.0      79.0
compsysmachardware        17.0      4.0      79.0
compwindowsx              21.0      2.0      77.0
miscforsale                8.0     10.0      82.0
recautos                  26.0      3.0      71.0
recmotorcycles            32.0      1.0      67.0
recsportbaseball          24.0      3.0      73.0
recsporthockey            28.0      3.0      69.0
scicrypt                  22.0      2.0      76.0
scielectronics            13.0      6.0      81.0
scimed                    28.0      2.0      70.0
scispace                  28.0      5.0      67.0
socreligionchristian      27.0      1.0      72.0
talkpoliticsguns          65.0      5.0      30.0
talkpoliticsmideast       68.0      NaN      32.0


In [21]:
# Evaluate the performance of your Naive Bayes classifier using metrics such as accuracy, precision, recall, and F1-score.

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print("F1-score:", f1)


Accuracy: 0.82
Precision: 0.844747244324381
Recall: 0.82
F1-score: 0.8144501134827706


In [23]:
# Discuss the performance of the model and any challenges encountered during the classification process.

# Model Performance Discussion

print("Model Performance Analysis:")
print("-" * 30)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Challenges encountered during the classification process
print("\nChallenges encountered during the classification process:")
print("-" * 30)

# 1. Data Preprocessing:
print("1. Data Preprocessing:")
print("   - Cleaning and preprocessing the text data (removing punctuation, stop words, etc.) can be challenging. "
      "Inconsistent formatting, slang, and abbreviations can make it difficult to standardize the data.")
print("   - Handling missing values and ensuring data quality are important for model performance.")

# 2. Feature Extraction:
print("\n2. Feature Extraction:")
print("   - Choosing the appropriate feature extraction method (TF-IDF) can impact the model's accuracy. "
      "Determining the optimal parameters for the vectorizer can be challenging and requires experimentation.")
print("   - The number of features can also influence performance. If too many features are used, it can lead to overfitting.")

# 3. Model Selection and Training:
print("\n3. Model Selection and Training:")
print("   - Selecting the right classifier (Multinomial Naive Bayes) and optimizing its parameters is important for achieving good performance.")
print("   - The model might be sensitive to the size and distribution of the training data. If the training data is imbalanced, the model may be biased towards the majority class.")

# 4. Sentiment Analysis:
print("\n4. Sentiment Analysis:")
print("   - Accurately capturing the sentiment from text can be challenging. Sarcasm, irony, and context can be difficult for the sentiment analyzer to interpret correctly.")
print("   - The choice of sentiment lexicon (VADER) can also affect the sentiment analysis results.")


# 5. Evaluation:
print("\n5. Evaluation:")
print("   - Selecting appropriate evaluation metrics (accuracy, precision, recall, F1-score) for evaluating the performance of the model is crucial.")
print("   - The metrics should align with the specific objectives of the classification task.")

# 6. Generalizability:
print("\n6. Generalizability:")
print("   - It's essential to evaluate the model's performance on unseen data to ensure that it generalizes well beyond the training set.")

# 7. Interpretability:
print("\n7. Interpretability:")
print("   - Sometimes, understanding the model's decision-making process and interpreting its predictions can be challenging.")



# Summarize the overall model performance and challenges:
print("\nSummary of Model Performance and Challenges:")
print("-" * 30)

print("The model shows decent accuracy, precision, recall, and F1-score, suggesting it performs relatively well in classifying the blog categories.")
print("However, certain challenges remain, such as ensuring data quality, optimizing feature extraction, handling potential biases in the training data, and evaluating the model's generalizability.")
print("Addressing these challenges can help further improve the model's performance and refine its ability to effectively classify blog categories.")


Model Performance Analysis:
------------------------------
Accuracy: 0.8200
Precision: 0.8447
Recall: 0.8200
F1-Score: 0.8145

Challenges encountered during the classification process:
------------------------------
1. Data Preprocessing:
   - Cleaning and preprocessing the text data (removing punctuation, stop words, etc.) can be challenging. Inconsistent formatting, slang, and abbreviations can make it difficult to standardize the data.
   - Handling missing values and ensuring data quality are important for model performance.

2. Feature Extraction:
   - Choosing the appropriate feature extraction method (TF-IDF) can impact the model's accuracy. Determining the optimal parameters for the vectorizer can be challenging and requires experimentation.
   - The number of features can also influence performance. If too many features are used, it can lead to overfitting.

3. Model Selection and Training:
   - Selecting the right classifier (Multinomial Naive Bayes) and optimizing its parame

In [25]:
# 	Reflect on the sentiment analysis results and their implications regarding the content of the blog posts.

print("Reflection on Sentiment Analysis Results:")
print("-" * 30)

# Analyze the overall sentiment distribution.
print("\nOverall Sentiment Distribution:")
print(df['Sentiment'].value_counts(normalize=True) * 100)  # Percentage distribution

# Discuss the implications of the sentiment distribution.
print("\nImplications of Sentiment Distribution:")
if df['Sentiment'].value_counts(normalize=True).get('positive', 0) > 0.5:
  print("The majority of blog posts exhibit a positive sentiment, suggesting a generally optimistic or upbeat tone in the blog content.")
elif df['Sentiment'].value_counts(normalize=True).get('negative', 0) > 0.5:
  print("The majority of blog posts exhibit a negative sentiment, suggesting a generally pessimistic or critical tone in the blog content.")
else:
  print("The blog posts show a balanced distribution of positive, negative, and neutral sentiments.")

# Analyze the sentiment distribution across categories.
print("\nSentiment Distribution Across Blog Categories:")
print(sentiment_by_category_percentage)

# Discuss the implications of the sentiment distribution across categories.
print("\nImplications of Sentiment Distribution Across Categories:")
for label in sentiment_by_category.index:
  print(f"\nCategory: {label}")
  category_sentiments = sentiment_by_category.loc[label]
  dominant_sentiment = category_sentiments.idxmax()

  if dominant_sentiment == 'positive':
    print("This category tends to have a positive sentiment, indicating that blog posts within this category might focus on uplifting or encouraging themes.")
  elif dominant_sentiment == 'negative':
    print("This category tends to have a negative sentiment, indicating that blog posts within this category might focus on critical, problem-oriented, or controversial topics.")
  else:
    print("This category tends to have a neutral sentiment, indicating that blog posts within this category might be more informative or objective in nature.")

# Provide further insights and interpretations of the sentiment analysis results.
print("\nFurther Insights:")
print(" - Examining the blog posts with the most extreme positive and negative sentiments can offer valuable insights into the specific topics, themes, and language that contribute to these sentiments.")
print(" - Understanding the interplay between sentiment and blog categories can help identify patterns and trends related to user engagement, audience preferences, and the overall tone of discussion within different communities.")
print(" - Analyzing sentiment over time can reveal shifts in opinions, attitudes, or trends within specific topics or communities.")


Reflection on Sentiment Analysis Results:
------------------------------

Overall Sentiment Distribution:
Sentiment
positive    67.80
negative    29.05
neutral      3.15
Name: proportion, dtype: float64

Implications of Sentiment Distribution:
The majority of blog posts exhibit a positive sentiment, suggesting a generally optimistic or upbeat tone in the blog content.

Sentiment Distribution Across Blog Categories:
Sentiment             negative  neutral  positive
Labels                                           
altatheism                40.0      1.0      59.0
compgraphics              10.0      3.0      87.0
composmswindowsmisc       22.0      2.0      76.0
compsysibmpchardware      18.0      3.0      79.0
compsysmachardware        17.0      4.0      79.0
compwindowsx              21.0      2.0      77.0
miscforsale                8.0     10.0      82.0
recautos                  26.0      3.0      71.0
recmotorcycles            32.0      1.0      67.0
recsportbaseball          24.0 